## 面向对象编程-高阶篇
1. __ slots __ (插槽)
2. 使用@property
3. 多重继承
4. 常用的类定义函数  
5. 枚举类
6. 元类

### 1. __ slots __
插槽函数，用来__限制__对实例的属性捆绑,也就是不能随便给实例添加属性

比如，只允许对Student实例添加name和age属性。

使用__ slots __ 要注意，__ slots __ 定义的属性仅对__当前类实例起作用__，对继承的子类是不起作用的

In [1]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

s = Student()
s.name = 'linthy'

In [13]:
# s.score = 99
# 将会出现 参数错误

### 2. @property
类似装饰器（decorator）  
有没有既能检查参数（实现函数功能），又可以用类似属性这样简单的方式（直接赋值）来访问类的变量呢？

In [12]:
class Student(object):

    def get_score(self):
         return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value
        
s = Student()
s.set_score(99)
print(s.get_score())
# 主动报错
# s.set_score(999)

99


提出问题，不要用函数，而是用属性实现。  
还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。  
Python内置的@property装饰器就是负责把一个方法变成属性调用的。

__见练习__  
@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值  

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。


__@property广泛应用在类的定义中，可以让调用者写出简短的代码，同时保证对参数进行必要的检查，这样，程序运行时就减少了出错的可能性。__

### 练习
请利用@property给一个Screen对象加上width和height属性，以及一个只读属性resolution：

In [5]:
class Screen(object):
    @property
    def width(self):
        return self._width
    @width.setter
    def width(self,value):
        if isinstance(value,int):
            self._width = value
    
    @property
    def height(self):
        return self._height
    @height.setter
    def height(self,value):
        if isinstance(value,int):
            self._height = value
    
    @property
    def resolution(self):
        return self._width * self._height
 # 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


### 3. 多重继承
为了给一个子类，继承更多父类的属性

__继承顺序，从左到右。采用去头法，以左为最先继承。__
具体参考：_https://www.jianshu.com/p/c9a0b055947b_

In [25]:
class Animal(object):
    def feature(self):
        print('It is an animal, is nor a plane.')

class Running(Animal):
    def featurn(self):
        print('It can run.')
    def gender(self):
        print('It have two gender, male or female.')

class Thinking(Animal):
    def feature(self):
        print('It can think.')

class People(Thinking, Running):
    pass

In [36]:
ren = People()
print(People.__mro__)
ren.feature()

(<class '__main__.People'>, <class '__main__.Thinking'>, <class '__main__.Running'>, <class '__main__.Animal'>, <class 'object'>)
It can think.


### 4.常用类定义函数
1. __ str __ ，用于打印函数
2. __ iter __ , 返回迭代器，结合__ next __ 一起用
3. __ getitem __ , 像list那样按照下标取出元素
4. __ getattr __ , 获取类内属性，若没有返回设定值 
5. __ call __

In [6]:
class Student(object):
    def __init__(self,name):
        self.name = name
    
    def __str__(self):
        return 'The student`s name is %s'% self.name
    # 这是因为直接显示变量调用的不是__str__()，而是__repr__()，
    # __str__()返回用户看到的字符串，而__repr__()返回程序开发者看到的字符串
    # 这里偷懒替换一下
    __repr__ = __str__

In [8]:
Student("linthy")

The student`s name is linthy

In [30]:
# 定义斐波那契数列Fib
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        if self.a > 100000: # 退出循环的条件
            raise StopIteration()
        return self.a
    
    # 加入getitem
    def __getitem__(self,n):
        for i in range(n):
            self.a, self.b = self.b, self.a + self.b
        return self.a

In [37]:
for i in Fib():
    print(i)
Fib(),Fib()[11]

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


(<__main__.Fib at 0x21b902bd9b0>, 89)

In [39]:
class Senior(Student):
    def __init__(self,name,subject):
        self.name = name
        self.subject = subject
    def __getattr__(self,attr):
        if attr == 'age':
            return 18

__若要获取的属性不存在__, 系统会报错  
要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个__getattr__()方法，动态返回一个属性。  
当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性，这样，我们就有机会返回score的值  

不单止可以返回值，还能返回函数  

In [41]:
class Chain(object):

    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

# 这个指令可以慢慢理解一下    
Chain().status.user.timeline.list

/status/user/timeline/list

#### Chain().status.user.timeline.list
urls = Chain()    # 初始化一个实例   
urls = urls.users    # 查找实例的一个属性  
urls = urls('michael)    # 调用一个函数  
urls = urls.repos    # 还是实例的属性  

### 5. 枚举类
为了避免全局常量被改动，定义一个class类型的枚举类，然后，每个常量都是class的一个唯一实例

In [46]:
from enum import Enum

Month = Enum('Month',('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'))


In [58]:
print(Month.__members__.items())
# value属性则是自动赋给成员的int常量，默认从1开始计数。

for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

odict_items([('Jan', <month.Jan: 1>), ('Feb', <month.Feb: 2>), ('Mar', <month.Mar: 3>), ('Apr', <month.Apr: 4>), ('May', <month.May: 5>), ('Jun', <month.Jun: 6>), ('Jul', <month.Jul: 7>), ('Aug', <month.Aug: 8>), ('Sep', <month.Sep: 9>), ('Oct', <month.Oct: 10>), ('Nov', <month.Nov: 11>), ('Dec', <month.Dec: 12>)])
Jan => month.Jan , 1
Feb => month.Feb , 2
Mar => month.Mar , 3
Apr => month.Apr , 4
May => month.May , 5
Jun => month.Jun , 6
Jul => month.Jul , 7
Aug => month.Aug , 8
Sep => month.Sep , 9
Oct => month.Oct , 10
Nov => month.Nov , 11
Dec => month.Dec , 12


如果需要更精确地控制枚举类型，可以从Enum派生出自定义类：

In [59]:
from enum import Enum, unique

#装饰器可以帮助我们检查保证没有重复值。
@unique
class Weekday(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [61]:
Weekday.__members__.items()

odict_items([('Sun', <Weekday.Sun: 0>), ('Mon', <Weekday.Mon: 1>), ('Tue', <Weekday.Tue: 2>), ('Wed', <Weekday.Wed: 3>), ('Thu', <Weekday.Thu: 4>), ('Fri', <Weekday.Fri: 5>), ('Sat', <Weekday.Sat: 6>)])

#### 练习
把Student的gender属性改造为枚举类型，可以避免使用字符串：

In [75]:
class Student(object):
    def __init__(self,name,gender):
        self._name = name
        self._gender = gender
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self,name):
        self._name = name
    @property
    def gender(self):
        return self._gender
    
    @gender.setter
    def gender(self,value):
        if isinstance(value, Gender):
            self._gender = value
        raise "The gender is error"
    
from enum import Enum, unique
@unique
class Gender(Enum):
    Male = 0
    Female = 1

# 测试:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


In [74]:
bart.gender


<Gender.Male: 0>

### 使用元类
动态语言和静态语言最大的不同，就是函数和类的定义，不是编译时定义的，而是运行时动态创建的。

除了使用type()动态创建类以外，要控制类的创建行为，还可以使用metaclass。  

metaclass，直译为元类

当我们定义了类以后，就可以根据这个类创建出实例，所以：先定义类，然后创建实例。

但是如果我们想创建出类呢？那就必须根据metaclass创建出类，所以：先定义metaclass，然后创建类。

连接起来就是：先定义metaclass，就可以创建类，最后创建实例。

所以，metaclass允许你创建类或者修改类。换句话说，你可以把类看成是metaclass创建出来的“实例”。

metaclass是Python面向对象里最难理解，也是最难使用的魔术代码。正常情况下，你不会碰到需要使用metaclass的情况，所以，以下内容看不懂也没关系，因为基本上你不会用到。

### 先跳过这个知识点